In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple, List


import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler


class KMeans:
    def __init__(self, n_clusters: int, max_iters: int = 100, random_state: int = None):
        """
        Initialize K-means clustering algorithm.
        
        Args:
            n_clusters: Number of clusters to form
            max_iters: Maximum number of iterations for the algorithm
            random_state: Random state for reproducibility
        """
        self.n_clusters = n_clusters
        self.max_iters = max_iters
        self.random_state = random_state
        self.centroids = None
        self.labels = None
        
    def fit(self, X: np.ndarray) -> 'KMeans':
        """
        Fit K-means clustering to the data.
        
        Args:
            X: Training data of shape (n_samples, n_features)
            
        Returns:
            self: Fitted KMeans instance
        """
        if self.random_state is not None:
            np.random.seed(self.random_state)
            
        # Randomly initialize centroids
        idx = np.random.choice(len(X), self.n_clusters, replace=False)
        self.centroids = X[idx]
        
        for _ in range(self.max_iters):
            # Assign points to nearest centroid
            old_labels = self.labels
            self.labels = self._assign_clusters(X)
            
            # Update centroids
            for k in range(self.n_clusters):
                if sum(self.labels == k) > 0:  # Only update if cluster has points
                    self.centroids[k] = X[self.labels == k].mean(axis=0)
            
            # Check for convergence
            if old_labels is not None and np.array_equal(old_labels, self.labels):
                break
                
        return self
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Predict cluster labels for new data.
        
        Args:
            X: New data of shape (n_samples, n_features)
            
        Returns:
            labels: Predicted cluster labels
        """
        return self._assign_clusters(X)
    
    def _assign_clusters(self, X: np.ndarray) -> np.ndarray:
        """
        Assign each point to the nearest centroid using Euclidean distance.
        
        Args:
            X: Data points of shape (n_samples, n_features)
            
        Returns:
            labels: Cluster assignments for each point
        """

        n_samples = len(X)
        distances = np.zeros((n_samples, self.n_clusters))
        
        # Calculate distances to each centroid
        for k in range(self.n_clusters):
            # Calculate squared distance between each point and current centroid
            diff = X - self.centroids[k]
            distances[:, k] = np.sum(diff * diff, axis=1)
        
        # Return index of closest centroid for each point
        return np.argmin(distances, axis=1)
        
        # # Calculate squared distances between each point and each centroid
        # # Step 1: Compute the difference between each data point and each centroid
        # differences = X[:, None, :] - self.centroids  # Shape: (n_samples, n_centroids, n_features)

        # # Step 2: Square the differences for all features
        # squared_differences = differences ** 2  # Shape: (n_samples, n_centroids, n_features)

        # # Step 3: Sum the squared differences across features to compute squared distances
        # distances = squared_differences.sum(axis=2)  # Shape: (n_samples, n_centroids) # Sum across features
        
        # return np.argmin(distances, axis=1)  # Find closest centroid for each point


def plot_clusters(X, kmeans, title):
    """Helper function to plot clusters and centroids"""
    plt.figure(figsize=(10, 6))
    plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels, cmap='viridis', alpha=0.7)
    plt.scatter(
        kmeans.centroids[:, 0],
        kmeans.centroids[:, 1],
        c='red',
        marker='x',
        s=200,
        linewidth=3,
        label='Centroids'
    )
    plt.title(title)
    plt.legend()
    plt.colorbar()
    plt.show()

def main():
    # Set random seed for reproducibility
    np.random.seed(42)

    # 1. Simple blob-like clusters
    print("Generating and clustering blob-like data...")
    X_blobs, _ = make_blobs(
        n_samples=300,
        centers=4,
        cluster_std=0.60,
        random_state=0
    )
    
    kmeans_blobs = KMeans(n_clusters=4, random_state=42)
    kmeans_blobs.fit(X_blobs)
    plot_clusters(X_blobs, kmeans_blobs, "K-means on Blob Data")

    # 2. Concentric circles
    print("\nGenerating and clustering circular data...")
    X_circles, _ = make_circles(n_samples=300, noise=0.05, factor=0.5)
    kmeans_circles = KMeans(n_clusters=2, random_state=42)
    kmeans_circles.fit(X_circles)
    plot_clusters(X_circles, kmeans_circles, "K-means on Circular Data")

    # 3. Moon-shaped clusters
    print("\nGenerating and clustering moon-shaped data...")
    X_moons, _ = make_moons(n_samples=300, noise=0.05)
    kmeans_moons = KMeans(n_clusters=2, random_state=42)
    kmeans_moons.fit(X_moons)
    plot_clusters(X_moons, kmeans_moons, "K-means on Moon Data")

    # 4. Random clusters with different variances
    print("\nGenerating and clustering random data with different variances...")
    random_centers = [(0, 0), (5, 5), (-3, 3)]
    X_random = []
    for cx, cy in random_centers:
        cluster = np.random.randn(100, 2) * np.random.uniform(0.3, 1.5)
        cluster += np.array([cx, cy])
        X_random.append(cluster)
    X_random = np.vstack(X_random)
    
    # Scale the data
    scaler = StandardScaler()
    X_random_scaled = scaler.fit_transform(X_random)
    
    kmeans_random = KMeans(n_clusters=3, random_state=42)
    kmeans_random.fit(X_random_scaled)
    plot_clusters(X_random_scaled, kmeans_random, "K-means on Scaled Random Data")

   

if __name__ == "__main__":
    main()